<a href="https://colab.research.google.com/github/LeeSeungYun1020/HandsOnLargeLanguageModels/blob/main/handsOnLLM03.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 대규모 언어 모델 개요

핸즈온 LLM 3장
- 트랜스포머 모델
- 트랜스포머 아키텍처의 발전


# 트랜스포머 모델

## 훈련된 트랜스포머 LLM의 입출력
- 한 번에 하나의 토큰 생성
- 정방향 계산: 프롬프트 -> 모델에 전달 -> 다음 토큰 생성 -> 출력된 토큰을 프롬프트에 추가
  - 모든 단계를 순서대로 한 번씩 수행
- 자기회귀 모델: 이전 예측 활용하여 다음 예측

## 정방향 계산의 구성 요소
- 토크나이저, 트랜스포머 블록의 스택, 언어 모델링(LM) 헤드로 구성
  - 토크나이저: 토큰의 테이블 어휘사전 포함, 모델은 각 토큰에 해당하는 임베딩(벡터 표현) 가짐
    - 텍스트 -> 토큰 -> 토큰 임베딩
  - 트랜스포머 블록 스택: 트랜스포머 블록 여러 개 순서대로 통과
  - LM 헤드: 다음 토큰 확률로 변환, 다른 헤드를 붙일 수 있음(Ex: 시퀀스 분류 헤드, 토큰 분류 헤드)

## 확률 분포에서 하나의 토큰 선택하기
- 디코딩 전략: 디코딩 과정에서 얻은 확률 분포에서 토큰 하나를 선택하는 방법, 확률 점수 기반 확률 분포에서 샘플링 수행 아이디어
  - 탐욕적 디코딩: 항상 확률이 가장 높은 토큰을 선택
  - 무작위성을 가미하거나 2, 3번째 토큰을 선택하는 것이 더 나은 경우도 있음 -> 6장

## 병렬 토큰 처리와 문맥 크기
- 트랜스포머는 기존 신경망 구조 대비 병렬 처리 향상
  - 입력 토큰을 개별 처리 스트림(트랙)으로 보내 처리
  - 모델 문맥 크기 = 모델이 다루는 최대 토큰 수 = 스트림 개수

## 키와 값을 캐싱하여 생성 속도 향상
- 키와 값 캐시: 어텐션 메커니즘에서 이전 스트림 출력을 사용 -> 이전 스트림 결과를 캐싱

## 트랜스포머 블록 내부
- 피드포워드 신경망: 모델 처리 용량의 대부분 담당
  - 정보 저장, 예측 생성, 데이터 보간
- 셀프 어텐션: 다른 입력 토큰 위치 정보 통합
  - 문맥 통합, 패턴(뉘앙스) 파악
  - 이전 토큰이 현재 토큰과 관련도 점수를 매긴 뒤(관련성 점수 계산) 각 토큰에서 얻은 정보를 출력 벡터에 통합(정보 통합)
  - 어텐션 헤드: 병렬로 실행되는 어텐션 메커니즘


# 트랜스포머 아키텍처의 발전

## 효율적인 어텐션
- 로컬/희소 어텐션: 주의를 기울일 이전 토큰 문맥 길이 제한
- 멀티 쿼리 어텐션: 키와 값 행렬을 모든 헤드에서 공유, 쿼리 행렬만 고유
- 그룹 쿼리 어텐션: 키와 값 행렬을 그룹 단위로 공유(품질 향상 위해 메모리 추가 사용), 쿼리 행렬은 고유
- 플래시 어텐션: GPU의 SRAM(공유 메모리)과 HBM(고대역폭 메모리) 사이 이동을 최적화하여 계산 속도 향상

## 트랜스포머 블록
- 선정규화: 어텐션, 피드포워드 과정 이전에 정규화 수행(기존에는 어텐션, 피드포워드 이후에 정규화 수행)
- RMS 정규화: 레이어 정규화를 간소화

## 위치 임베딩
- 모델이 시퀀스 안에서 토큰의 순서를 추적할 수 있는 정보
- 로터리 위치 임베딩: 상대적인 토큰 위치 정보를 인코딩
  - 훈련 데이터에 여러 문서 포함할 수 있어 대규모 모델 훈련에서 효율성 향상

## 실험적 구조와 개선 사항
- 컴퓨터 비전, 로봇공학, 시계열 분야에서 트랜스포머 구조 적용 및 연구 중



In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("microsoft/Phi-3-mini-4k-instruct")

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

added_tokens.json:   0%|          | 0.00/306 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/599 [00:00<?, ?B/s]

In [ ]:
model = AutoModelForCausalLM.from_pretrained(
    "microsoft/Phi-3-mini-4k-instruct",
    device_map="cuda",
    torch_dtype="auto",
    trust_remote_code=False,
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
generator = pipeline(
    "text-generation",
    model = model,
    tokenizer = tokenizer,
    return_full_text = False,
    max_new_token = 128,
    do_sample = False,
)

Device set to use cuda
The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


In [ ]:
prompt = "Write an email apologizing to Yun for the tragic gardening mishap. Explain how it happened."
output = generator(prompt, do_sample = False, use_cache=False)

The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


In [ ]:
print(output[0]["generated_text"])

 Express your regret and offer to help with the garden repairs.

Subject: Sincere Apologies for the Gardening Mishap

Dear Yun,

I hope this email finds you well. I am writing to express my deepest apologies for the unfortunate incident that occurred in your garden recently. It was truly a tragic mishap, and I am truly sorry for any distress or inconvenience it may have caused you.

The incident happened when I was attempting to help you with your gardening project. Unfortunately, due to my lack of experience and knowledge in this area, I unintentionally caused some damage to your plants. I understand that this was not my intention, and I deeply regret the outcome.

I realize that my actions have caused you a great deal of frustration and disappointment. Please know that I am truly sorry for any inconvenience this may have caused you. I take full responsibility for my actions and understand that it was my mistake.

To make amends, I would like to offer my assistance in repairing the da

In [ ]:
print(model)

Phi3ForCausalLM(
  (model): Phi3Model(
    (embed_tokens): Embedding(32064, 3072, padding_idx=32000)
    (embed_dropout): Dropout(p=0.0, inplace=False)
    (layers): ModuleList(
      (0-31): 32 x Phi3DecoderLayer(
        (self_attn): Phi3Attention(
          (o_proj): Linear(in_features=3072, out_features=3072, bias=False)
          (qkv_proj): Linear(in_features=3072, out_features=9216, bias=False)
          (rotary_emb): Phi3RotaryEmbedding()
        )
        (mlp): Phi3MLP(
          (gate_up_proj): Linear(in_features=3072, out_features=16384, bias=False)
          (down_proj): Linear(in_features=8192, out_features=3072, bias=False)
          (activation_fn): SiLU()
        )
        (input_layernorm): Phi3RMSNorm()
        (resid_attn_dropout): Dropout(p=0.0, inplace=False)
        (resid_mlp_dropout): Dropout(p=0.0, inplace=False)
        (post_attention_layernorm): Phi3RMSNorm()
      )
    )
    (norm): Phi3RMSNorm()
  )
  (lm_head): Linear(in_features=3072, out_features=3206

In [ ]:
prompt = "The most famous fruit is the"
input_ids = tokenizer(prompt, return_tensors="pt").input_ids
input_ids = input_ids.to("cuda")
model_output = model.model(input_ids, use_cache=False)
lm_head_output = model.lm_head(model_output[0])

In [ ]:
token_id = lm_head_output[0,-1].argmax(-1)
tokenizer.decode(token_id)

'apple'

In [ ]:
for out in lm_head_output[0]:
  print(tokenizer.decode(out.argmax(-1)))

code
common
example
of
the
apple


In [ ]:
model_output[0].shape

torch.Size([1, 6, 3072])

In [ ]:
lm_head_output.shape

torch.Size([1, 6, 32064])

In [ ]:
prompt = "The capital city of Korea is"
for i in range(1, 20):
  input_ids = tokenizer(prompt, return_tensors="pt").input_ids
  input_ids = input_ids.to("cuda")
  model_output = model.model(input_ids, use_cache=False)
  lm_head_output = model.lm_head(model_output[0])
  token_id = lm_head_output[0,-1].argmax(-1)
  print(tokenizer.decode(token_id))
  prompt = prompt + tokenizer.decode(token_id)

# Seoul은 두 단어로 나누어 토큰화되어 있음

Se
oul
.




##
#
Message
:


Trans
late
to
English
:




##
#


In [ ]:
prompt = "Explain to me about the capital city of Korea."
input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to("cuda")

In [ ]:
%%timeit -n 1
out = model.generate(
    input_ids = input_ids,
    max_length = 128,
    use_cache = False
)
print(out)

tensor([[12027,  7420,   304,   592,  1048,   278,  7483,  4272,   310, 19109,
         29889,    13,    13,  1068, 13296,   918, 29871, 29896, 29901,  1068,
            13,    13,  1576,  7483,  4272,   310, 19109,   338,   922,  5059,
         29889,   922,  5059,   338,   278, 10150, 25311,   275,   297,  4275,
         19109,   322, 19700,   408,   278,  8604, 29892, 16375, 29892,   322,
         17407,  4818,   310,   278,  4234, 29889,   739,   338,  5982,   297,
           278,  6641,  5933,   760,   310,   278,  4234, 29892,  2978,   278,
          1261,   309,  3673,  1891, 10640,   313, 23560, 29999, 29897,   393,
          2903,  1078,   372,   515,  4644, 19109, 29889,    13,    13,  2008,
          5059,   338,  2998,   363,   967,  5400,  2071,   952, 29883,  2390,
           414, 29892,  1880, 29899, 11345,  1014,  1994, 29892,   322,  1835,
          9257, 29892,  3704,   476, 29899,  7323,   322, 22467,  8541,   294,
         29889,   450,  4272,   338,   884,  3271,  

In [ ]:
%%timeit -n 1
out = model.generate(
    input_ids = input_ids,
    max_length = 128,
    use_cache = True
)
print(out)

tensor([[12027,  7420,   304,   592,  1048,   278,  7483,  4272,   310, 19109,
         29889,    13,    13,  1068, 13296,   918, 29871, 29896, 29901,  1068,
            13,    13,  1576,  7483,  4272,   310, 19109,   338,   922,  5059,
         29889,   922,  5059,   338,   278, 10150, 25311,   275,   297,  4275,
         19109,   322, 19700,   408,   278,  8604, 29892, 16375, 29892,   322,
         17407,  4818,   310,   278,  4234, 29889,   739,   338,  5982,   297,
           278,  6641,  5933,   760,   310,   278,  4234, 29892,  2978,   278,
          1261,   309,  3673,  1891, 10640,   313, 23560, 29999, 29897,   393,
          2903,  1078,   372,   515,  4644, 19109, 29889,    13,    13,    13,
          2008,  5059,   338,  2998,   363,   967,  5400,  2071,   952, 29883,
          2390,   414, 29892,  1880, 29899, 11345,  1014,  1994, 29892,   322,
          1835,  9257, 29892,  3704,   476, 29899,  7323,   322, 22467, 24615,
         29889,   450,  4272,   338,   884,  3271,  